# 📌 Understanding `Annotated` in Python

## 🔹 What is `Annotated`?
`Annotated` is a feature introduced in **Python 3.9** (backported in `typing_extensions` for older versions). It allows us to attach **metadata** to type hints, which can be used by tools like **Pydantic** or static type checkers to enforce additional constraints.

### ✅ Basic Syntax
```python
from typing import Annotated

CustomType = Annotated[int, "Some metadata"]
```

This does not change the actual type (int in this case) but adds extra metadata that external tools can use.

In [ ]:
from typing import Annotated

def process_age(age: Annotated[int, "Age must be a positive integer"]) -> str:
    return f"Age: {age}"

print(process_age(25))  # Output: Age: 25

Here, the **metadata** `"Age must be a positive integer"` is added but does not enforce validation—it is just a hint for developers or tools.

# 🔍 Understanding `Annotated` in Python

The `Annotated` type hint in Python is a powerful tool that allows us to attach metadata, constraints, and validation logic to types. While many think `Annotated` is only for metadata, it can also be used with **Pydantic validation**, `Field`, and even custom functions.

---

## 🚀 Why Use `Annotated`?  

`Annotated` enhances type hints by allowing additional information that tools and libraries (like Pydantic) can use for validation, constraints, or documentation.

---

## 🔥 Beyond Metadata: Other Uses of `Annotated`

### 2 Using `Annotated` with `Field`  
In **Pydantic**, we can use `Field` to add validation and constraints.

```python
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    age: Annotated[int, Field(gt=18, description="Age must be greater than 18")]

# ✅ Valid: User(age=25)
# ❌ Invalid: User(age=17) -> Raises validation error

In [ ]:
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    age: Annotated[int, Field(gt=0, lt=120)]  # Age must be between 1 and 119

user = User(age=25)  # ✅ Works fine

In [ ]:
user = User(age=150)  # ❌ Raises validation error

### 3. Using Annotated with Custom Functions

In [ ]:
from typing import Annotated

def validate_age(value: int) -> int:
    if value < 0:
        raise ValueError("Age cannot be negative")
    return value

AgeType = Annotated[int, validate_age]  # ❌ This alone does nothing at runtime

def set_age(age: int) -> str:
    age = validate_age(age)  # ✅ Explicitly apply validation
    return f"User age is {age}"

print(set_age(-5))  # ❌ Raises ValueError: "Age cannot be negative"

🔥 Key Takeaways
✔ `Annotated` **does not enforce rules by itself** but provides metadata for validation tools.
✔ Can be used with `pydantic`, `Field`, or custom validation functions.
✔ Helps in making `type hints more descriptive` for documentation and static analysis.

In [ ]:
import annotated_types
PositiveInt = Annotated[int, annotated_types.Gt(0)]


## 🔥 Using `Annotated` with `annotated_types`

`annotated_types` provides constraints like:

- **`Gt(value)`**: Greater than a specific value  
- **`Ge(value)`**: Greater than or equal to a value  
- **`Lt(value)`**: Less than a specific value  
- **`Le(value)`**: Less than or equal to a value  
- **`Interval(ge=a, le=b)`**: Restrict values within a range  

In [ ]:
from typing import Annotated
from annotated_types import Gt  # Gt (Greater than) enforces a value > 0

PositiveInt = Annotated[int, Gt(0)]

def process_age(age: PositiveInt) -> str:
    return f"User age: {age}"

print(process_age(25))  # ✅ Works fine

In [ ]:
print(process_age(-5))  # ❌ Raises an error if validated using a framework

In [ ]:
from pydantic import BaseModel
from typing import Annotated
from annotated_types import Gt

class User(BaseModel):
    age: Annotated[int, Gt(0)]  # Ensures age is > 0

user = User(age=30)  # ✅ Works
user.age

In [ ]:
user = User(age=-5)  # ❌ Raises validation error

In [ ]:
from typing import Annotated
from annotated_types import Gt, Le

PositiveInt = Annotated[int, Gt(0)]
LimitedFloat = Annotated[float, Gt(0), Le(100)]  # Value must be between 0 and 100
class Score(BaseModel):
    score : PositiveInt

Score(score=85.5)


In [ ]:
from typing import Annotated
from annotated_types import Gt, Le
from pydantic import BaseModel

PositiveInt = Annotated[int, Gt(0)]
LimitedFloat = Annotated[float, Gt(0), Le(100)]  # Float between 0 and 100

class Score(BaseModel):
    score: LimitedFloat  # ✅ Accepts both int and float in range

print(Score(score=85.5))  # ✅ Works fine

In [ ]:
print(Score(score=150))   # ❌ Raises ValidationError (out of range)
# print(Score(score=-5))    # ❌ Raises ValidationError (negative)

## Pydantic Revision

In [ ]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

user = User(**external_data)  

print(user.id)  
print(user.model_dump())  

If validation fails, Pydantic will raise an error with a breakdown of what was wrong:

In [ ]:
# continuing the above example...

from datetime import datetime
from pydantic import BaseModel, PositiveInt, ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}  

try:
    User(**external_data)
except ValidationError as e:
    print(e.json(indent=2))  

In [10]:
from typing import List
def sort_numbers(numbers: List[int]) -> List[int]:
    my_list, num = [], min(numbers)
    while numbers:
        num = min(numbers)
        my_list.append(num)
        numbers.remove(num)
    return my_list

print(sort_numbers([1, 5, 3, 2, 4, 11, 19, 9, 2, 5, 6, 7, 4, 2, 6]))

[1, 2, 2, 2, 3, 4, 4, 5, 5, 6, 6, 7, 9, 11, 19]


In [19]:
def sort_decimals(numbers: List[float]) -> List[float]:
    sorted_decimals = []
    while numbers:
        smallest = numbers[0]
        for number in numbers:
            if number < smallest:
                smallest = number
        sorted_decimals.append(smallest)
        numbers.remove(smallest)
    return sorted_decimals

print(sort_decimals([3.14, 2.82, 6.433, 7.9, 21.555, 21.554]))

[2.82, 3.14, 6.433, 7.9, 21.554, 21.555]


In [ ]:
def sort_numbers(numbers: List[int]) -> List[int]:
    my_list, num = [], min(numbers)
    while numbers:
        num = min(numbers)
        my_list.append(num)
        numbers.remove(num)
    return my_list

print(sort_numbers([1, 5, 3, 2, 4, 11, 19, 9, 2, 5, 6, 7, 4, 2, 6]))

In [12]:
def sort_words(words: List[str]) -> List[str]:
    sorted_list = []
    while words:
        smallest = words[0]
        for word in words:
            if word < smallest:
                smallest = word
        sorted_list.append(smallest)
        words.remove(smallest)
    return sorted_list
print(sort_words(["cherry", "apple", "blueberry", "banana", "watermelon", "zucchini", "kiwi", "pear"]))


['apple', 'banana', 'blueberry', 'cherry', 'kiwi', 'pear', 'watermelon', 'zucchini']


In [ ]:
from typing import List


def sort_words(words: List[str]) -> List[str]:
    sorted_list = []
    while words:
        smallest = words[0]
        for word in words:
            if word < smallest:
                smallest = word
        sorted_list.append(smallest)
        words.remove(smallest)
    return sorted_list

def sort_numbers(numbers: List[int]) -> List[int]:
    my_list, num = [], min(numbers)
    while numbers:
        num = min(numbers)
        my_list.append(num)
        numbers.remove(num)
    return my_list

def sort_decimals(numbers: List[float]) -> List[float]:
    my_list, num = [], min(numbers)
    while numbers:
        num = min(numbers)
        my_list.append(num)
        numbers.remove(num)
    return my_list



# do not modify below this line
print(sort_words(["cherry", "apple", "blueberry", "banana", "watermelon", "zucchini", "kiwi", "pear"]))

print(sort_numbers([1, 5, 3, 2, 4, 11, 19, 9, 2, 5, 6, 7, 4, 2, 6]))

print(sort_decimals([3.14, 2.82, 6.433, 7.9, 21.555, 21.554]))
